In [1]:
from core_components.maps.tilemaps import GenericTileMap
import numpy as np
a = GenericTileMap((10,10))
a.add_area(np.s_[2:5, 2:5], tile_type="wall")

In [ ]:
def istype(a: np.ndarray, dtype: np.dtype) -> bool:
    """Helper function to check if a numpy array is of a specific dtype"""
    if a.dtype.metadata is None or dtype.metadata is None:
        return False
    return a.dtype.metadata['__name__'] == dtype.metadata['__name__']

ascii_default = np.array((ord(' '), (255, 255, 255), (0, 0, 0)), dtype=ascii_graphic)

istype(ascii_default, ascii_graphic)  # True

In [ ]:
def run_tests(test_list):
    count = 0
    for test in test_list:
        try:
            test()
            count += 1
        except Exception as e:
            print(f"Exception: {test.__name__}: {e}")
            continue
    print(f"{count} of {len(test_list)} tests ran successfully.")

In [ ]:
from test.test_component_actions_system_dispatcher import *
tests = [test_dispatcher_ev_gamestart,
         test_dispatcher_ev_gameover,
         test_dispatcher_ev_keydown]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_component_actions_dispatchers import *
tests = [test_dispatcher_ev_quit,
         test_dispatcher_initialization]

run_tests(tests)
# --- IGNORE ---

In [ ]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



In [ ]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [ ]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()
